In [ ]:
!pip install ultralytics ffmpeg-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.7/914.7 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

In [ ]:
import os
import cv2
import logging
import ffmpeg
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# Initialize logging
logging.basicConfig(level=logging.INFO)

# Define function to check prerequisites
def check_prerequisites(output_text_file, frames_folder, annotations_folder):
    try:
        # Clear previous log file
        with open(output_text_file, "w", encoding="utf-8"):
            pass
        logging.info(f"Eski {output_text_file} dosyasının içeriği silindi.")
    except Exception as e:
        logging.exception(f"Log dosyası sıfırlanırken hata oluştu: {e}")

    prerequisites_action(frames_folder)
    prerequisites_action(annotations_folder)

def prerequisites_action(action_parameter):
    if not os.path.exists(action_parameter):
        os.makedirs(action_parameter)
        logging.info(f"{action_parameter} klasörü oluşturuldu.")
    else:
        # Clean the folder by removing only files, keeping subdirectories
        for item in os.listdir(action_parameter):
            item_path = os.path.join(action_parameter, item)
            if os.path.isfile(item_path) or os.path.islink(item_path):
                try:
                    os.unlink(item_path)  # Remove files and symlinks
                    logging.info(f"{item_path} silindi.")
                except Exception as e:
                    logging.error(f"Silme hatası: {item_path} - {e}")

def save_processed_frames(ret, frame_count, output_folder, frame):
    if ret:
        output_path = os.path.join(output_folder, f"frame_{frame_count}.jpg")
        cv2.imwrite(output_path, frame)
        logging.info(f"Frame {frame_count} kaydedildi: {output_path}")
    else:
        logging.warning("Kare bulunamadı veya kaydedilemedi.")


def save_processed_txts(annotation_lines, annotations_folder, frame_count):
    # Ensure the annotations folder exists
    os.makedirs(annotations_folder, exist_ok=True)

    # Construct the full path to save the file inside the annotations folder
    file_path = os.path.join(annotations_folder, f"frame_{frame_count}.txt")

    # Write the annotations to the file
    with open(file_path, "w") as outfile:
        for line in annotation_lines:
            outfile.write(line + "\n")

    logging.info(f"Etiket {frame_count} kaydedildi: {file_path}")

def save_detected_objects_txt(frame_count, frame, results_list, width, height):
    annotation_lines = []
    for obj in results_list:
        xmin, ymin, xmax, ymax = obj[0], obj[1], obj[2], obj[3]
        xmin = max(0, min(xmin, width))
        ymin = max(0, min(ymin, height))
        xmax = max(0, min(xmax, width))
        ymax = max(0, min(ymax, height))
        if xmin >= xmax or ymin >= ymax:
            continue

        center_x = float((xmin + xmax) / 2) / width
        center_y = float((ymin + ymax) / 2) / height
        box_width = float(xmax - xmin) / width
        box_height = float(ymax - ymin) / height

        yolo_annotations = f"{obj[5]} {center_x} {center_y} {box_width} {box_height}"
        annotation_lines.append(yolo_annotations)

    save_processed_txts(annotation_lines,'/content/annotations',frame_count)


In [ ]:
#uploaded_video='/content/drive/MyDrive/pexels_araç_videoları/video_1.mp4'
#uploaded_video='/content/drive/MyDrive/pexels_araç_videoları/video_10.mp4'
uploaded_video='/content/drive/MyDrive/pexels_araç_videoları/video_11.mp4'


In [ ]:

# Check prerequisites
check_prerequisites('terminal.txt', 'frames', 'annotations')

CONFIDENCE_THRESHOLD = 0.45
cap = cv2.VideoCapture(uploaded_video)
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
duration = total_frames / fps

# Get video metadata
probe = ffmpeg.probe(uploaded_video)
video_info = next(stream for stream in probe["streams"] if stream["codec_type"] == "video")
width = int(video_info["width"])
height = int(video_info["height"])

frame_interval = int(fps/2) if duration <= 20 else int(fps)

# Load YOLO model
model = YOLO("/content/drive/MyDrive/best.pt")

frame_count = 0  # Initialize frame counter
while True:
    ret, frame = cap.read()
    if not ret:
        logging.warning("Video bitti veya frame okunamadı.")
        break

    detections = model(frame, device='cuda')[0]
    results_list = []

    for data in detections.boxes.data.tolist():
        confidence = data[4]
        if float(confidence) < CONFIDENCE_THRESHOLD:
            continue
        xmin, ymin, xmax, ymax = int(data[0]), int(data[1]), int(data[2]), int(data[3])
        class_id = int(data[5])
        results_list.append([xmin, ymin, xmax, ymax, confidence, class_id])

    if frame_count % frame_interval == 0:
      save_processed_frames(ret, frame_count, 'frames', frame)
      save_detected_objects_txt(frame_count, frame, results_list, width, height)

    frame_count += 1

logging.info("Görüntü işleme sonlandı.")
cap.release()


0: 384x640 31 Arabas, 8 Insans, 64.1ms
Speed: 2.8ms preprocess, 64.1ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 31 Arabas, 8 Insans, 36.1ms
Speed: 2.7ms preprocess, 36.1ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 31 Arabas, 7 Insans, 35.7ms
Speed: 2.6ms preprocess, 35.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 32 Arabas, 7 Insans, 36.4ms
Speed: 2.4ms preprocess, 36.4ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 32 Arabas, 9 Insans, 36.2ms
Speed: 2.5ms preprocess, 36.2ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 32 Arabas, 9 Insans, 35.8ms
Speed: 2.3ms preprocess, 35.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 32 Arabas, 10 Insans, 35.8ms
Speed: 2.2ms preprocess, 35.8ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 32 Arabas, 8 Insans, 34.3ms


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
